# Placement

This is based on the work of "Architecture-aware FPGA placement using metric embedding"
The objective for a metric based placement will be: $||D_DP-PD_A||_F^2$ where P is the permutation matrix and $D_x$ is the distance matrix.
The constraints on permutation matrix are: $\sum_{i=1}^n P(i,j) = 1, \forall j$ 

and $\sum_{j=1}^n P(i,j) = 1, \forall i$

Casting the placement into QUBO:

-objective
$f=\sum_I^n\sum_J^n\{-2D_{II}A_{JJ}x_{IJ}+\sum_i^n[D_{Ii}^2x_{iJ}+A_{iJ}^2x_{Ii}+\sum_{j\neq i}^n (D_{Ii}D_{Ij}x_{iJ}x_{jJ}+A_{iJ}A_{jJ}x_{Ii}x_{Ij})]-\sum_{i\neq I}^n\sum_{j\neq J}^n[A_{jJ}D_{Ii}x_{iJ}x_{Ij}]-\sum_{i\neq J}^n\sum_{j\neq I}^n[A_{iJ}D_{Ij}x_{Ii}x_{jJ}]\}$

-constraint on row and columns separately (in case we need to relax one):
$\sum_j^n [1-\sum_i^nx_{ij}]^2=\sum_j^n\sum_i^n[-x_{ij}+2\sum_{k\neq i}^nx_{ij}x_{kj}]$

$\sum_i^n [1-\sum_j^nx_{ij}]^2=\sum_i^n\sum_j^n[-x_{ij}+2\sum_{k\neq j}^nx_{ij}x_{ik}]$

In [ ]:
#distance matrix calculator

In [ ]:
from essentials import *
from itertools import product as prd

In [ ]:
G = RectGridGraph(2, 3)
G.draw()

In [ ]:
D_D = np.array([[0, 2, 2, 1, 2, 2], [2, 0, 2, 1, 2, 2], [2, 2, 0, 1, 2, 2], [1, 1, 1, 0, 1, 1],[2, 2, 2, 1, 0, 2],[2, 2, 2, 1, 2, 0]])
D_A = np.array([[0, 1, 1, 2, 2, 3], [1, 0, 2, 1, 3, 2], [1, 2, 0, 1, 1, 2], [2, 1, 1, 0, 2, 1],[2, 3, 1, 2, 0, 1],[3, 2, 2, 1, 1, 0]])

In [ ]:
def set_qubo_val(Q, x1, x2, value):
    if (x1, x2) in Q.keys():
        Q[(x1, x2)] += value
    elif (x2, x1) in Q.keys():
        Q[(x2, x1)] += value
    else:
        raise Exception('Your key is not in the Q dict.') 
        
def placement_qubo(D_design, D_architecture, params={'weight_objective': 1, 'weight_row': 1, 'weight_column': 1}):
    #D_design and D_architecture matrices are 2-D numpy arrays
    Q = {}
    n = len(D_design)
    D = D_design
    A = D_architecture
    # for now the design and architecture distance matrices are of the same size
    
    permutation_matrix_elements=[]
    for var1, var2 in prd(range(0, n), repeat=2):
        permutation_matrix_elements.append([var1, var2])
        
    for var1, var2 in cwr(permutation_matrix_elements, 2):
            Q[(f'x{var1[0]}{var1[1]}', f'x{var2[0]}{var2[1]}')] = 0
    
    # Objective
    w1 = params['weight_objective']
    for I, J in prd(range(0,n), repeat=2):
        Q[(f'x{I}{J}', f'x{I}{J}')] += w1 * -2 * D[I][I] * A[J][J]
        for i in range(0,n):
            Q[(f'x{i}{J}', f'x{i}{J}')] += w1 * D[I][i]^2
            Q[(f'x{I}{i}', f'x{I}{i}')] += w1 * A[i][J]^2
            for j in range(0,n):
                if i != j:
                    set_qubo_val(Q, f'x{i}{J}', f'x{j}{J}', w1 * D[I][i] * D[I][j])
                    set_qubo_val(Q, f'x{I}{i}', f'x{I}{j}', w1 * A[i][J] * A[j][J])
        for i, j in prd(range(0,n), repeat=2):
            if i != I and j != J:
                set_qubo_val(Q, f'x{i}{J}', f'x{I}{j}', -w1 * A[j][J] * D[I][i])
            if i != J and j != I:
                set_qubo_val(Q, f'x{I}{i}', f'x{j}{J}', -w1 * A[i][J] * D[I][j])

    #constraint on rows
    w2 = params['weight_row']
    for i, j in prd(range(0, n), repeat=2):
        Q[(f'x{i}{j}', f'x{i}{j}')] += w2 * -1
        for k in range(0, n):
            if k != i:
                set_qubo_val(Q, f'x{i}{j}', f'x{k}{j}', w2 * 2)
    
    #constraint on columns
    w3 = params['weight_column']
    for i, j in prd(range(0, n), repeat=2):
        Q[(f'x{i}{j}', f'x{i}{j}')] += w3 * -1
        for k in range(0, n):
            if k != j:
                set_qubo_val(Q, f'x{i}{j}', f'x{i}{k}', w3 * 2) 
    return Q

In [ ]:
Q=placement_qubo(D_A, D_D)
dwave_sampler = DWaveSampler(solver={'lower_noise': True, 'qpu': True})
A = dwave_sampler.edgelist
embedding, chain_len = find_embedding_minorminer(Q, A)
## the shortest chain_len I've seen with num_tries=1000 is 5
## (SP: takes 2.5 mins on my machine, SAS: 1:08 on mine)
display(chain_len)

In [ ]:
connectivity_structure = dnx.chimera_graph(16,16)
fig=plt.figure(figsize=(25, 25))
dnx.draw_chimera_embedding(connectivity_structure, embedding)

In [ ]:
fixed_sampler = FixedEmbeddingComposite(
            DWaveSampler(solver={'lower_noise': True, 'qpu': True}), embedding
            )
response = optimize_qannealer(fixed_sampler, Q, params={'chain_strength': 20, 'annealing_time': 99, 'num_reads': 10000})
display(response.first)
best_q_answer = response.first.sample

# Distance matrix calculation

In [ ]:
# finding the distance matrix based on the edge list usinf BFS algorithm
import queue
def find_adjacent_edges(start_node, edge_list):
    adjacent_edges = []
    for item in edge_list:
        if start_node == item[0]:
            adjacent_edges.append(item)
        elif start_node == item[1]:
            adjacent_edges.append((item[1], item[0]))
    return adjacent_edges

def distance_calc(input_dict, start_node, end_node):
    temp = end_node
    distance = 0
    while temp != start_node:
        temp = input_dict[temp][0]['parent']
        distance += 1
    return distance

def BFS(node_list, edge_list, start_node, end_node):
    matrix_size = len(node_list)
    matrix_distance = np.zeros((matrix_size, matrix_size))
    q = queue.Queue()
    #FIFO queue
    q.put(start_node)
    d = {key : [{'parent' : None}, {'discovered' : False}] for key in node_list}
    distance = 0
    while not q.empty():
        current_node = q.get()
        if current_node == end_node:
            return distance_calc(d, start_node, current_node)
        neighbor_edgelist = find_adjacent_edges(current_node, edge_list)
        for item in neighbor_edgelist:
            if d[item[1]][1]['discovered'] == False:
                d[item[1]][1]['discovered'] = True
                d[item[1]][0]['parent'] = current_node
                q.put(item[1])

def distance_matrix(node_list, edge_list):
    dmatrix = np.zeros((len(node_list), len(node_list)))
    for i, j in prd(node_list, repeat=2):
        dmatrix[i][j] = BFS(node_list, edge_list, i, j)
    return dmatrix

def partial_trace(input_matrix, dim1, dim2):
    #computes partial trace of dim1 kron dim2
    reshaped_mat = input_matrix.reshape([dim1, dim2, dim1, dim2])
    reduced_1 = np.einsum('kjij->ki', reshaped_mat)
    reduced_2 = np.einsum('jkji->ki', reshaped_mat)
    return [reduced_1, reduced_2]

In [ ]:
#Testing the trace function
a=np.kron(np.eye(2,2), np.diag([i for i in range(0,8)])) 
[reduced_1, reduced_2] = partial_trace(a, 2, 8);
reduced_2

In [ ]:
node_list = [i for i in range(0,8)]
edge_list =[(0, 4),
 (0, 5),
 (0, 6),
 (0, 7),
 (1, 4),
 (1, 5),
 (1, 6),
 (1, 7),
 (2, 4),
 (2, 5),
 (2, 6),
 (2, 7),
 (3, 4),
 (3, 5),
 (3, 6),
 (3, 7)]
d = distance_matrix(node_list, edge_list)

In [ ]:
#w, v = np.linalg.eig(d)
d

In [ ]:
connectivity_structure = dnx.chimera_graph(2,1)

In [ ]:
node_list = list(connectivity_structure.nodes)
edge_list = list(connectivity_structure.edges)
d1 = distance_matrix(node_list, edge_list)
d2 = np.kron(np.eye(2, 2), d)
traced_matrix = partial_trace(d1, 2, 8);

In [ ]:
dwave_sampler = DWaveSampler(solver={'lower_noise': True, 'qpu': True})
A = dwave_sampler.edgelist
node_list = [i for i in range(0,2048)]
edge_list = A # Dwave QPO edge list
d = distance_matrix(node_list, edge_list)

In [ ]:
d